In [ ]:
import tensorflow as tf  # for deep learning
from residual_layer import ResidualLayer2D # for Residual layer
# from data_handler import data_loader_csv
import os
import time 

In [ ]:
# create cifar dataset
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.cifar10.load_data()
train_data = tf.data.Dataset.from_tensor_slices((tf.cast(X_train, tf.float32),tf.squeeze(tf.cast(Y_train, tf.float32)))).shuffle(50000).batch(64)
test_data = tf.data.Dataset.from_tensor_slices((tf.cast(X_test, tf.float32),tf.squeeze(tf.cast(Y_test, tf.float32)))).shuffle(10000).batch(64)

170500096/170498071 [==============================] - 4s 0us/step


In [ ]:
# create a basic model
class Residual(tf.keras.Model):
    def __init__(self, classes, *args, **kwargs):
        self.classes = classes
        super(Residual, self).__init__(*args, **kwargs)

        self.model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(filters=32,
                                   kernel_size=3,
                                   padding="same",),
            ResidualLayer2D(filters=32),
            ResidualLayer2D(filters=32),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation=tf.nn.relu),
            tf.keras.layers.Dropout(rate=0.3),
            tf.keras.layers.Dense(self.classes)
        ])

    def call(self, inputs):
        return self.model(inputs / 255.0)

model = Residual(10)

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(), loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit_generator(train_data, epochs=5, validation_data=test_data)